In [1]:
!pip3 install fasttext

In [2]:
# https://fasttext.cc/docs/en/references.html#bag-of-tricks-for-efficient-text-classification

# https://www.geeksforgeeks.org/fasttext-working-and-implementation/#:~:text=FastText%20is%20very%20fast%20in,classifier%20to%20train%20the%20model.

In [92]:
import fasttext
import re

import pandas as pd

from sklearn.model_selection import train_test_split
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from sklearn.metrics import classification_report

In [182]:
def clean_sentence(sentence, remove_stop_words=False):
    tokens = simple_preprocess(sentence, max_len=30)
    
    if remove_stop_words:
        tokens = [t for t in tokens if t not in STOPWORDS]
        
    return ' '.join(tokens)

In [194]:
df = pd.read_csv('../data/obligation_extraction_df.csv')
df_train, df_test = train_test_split(df, train_size=0.7, random_state=42)

In [195]:
df_train['cleaned_sentence'] = df_train.sentence.apply(lambda x: clean_sentence(x, True))
df_train['label'] = df_train.is_obligation.apply(lambda x: f'__label__{str(x)}')

df_test['cleaned_sentence'] = df_test.sentence.apply(lambda x: clean_sentence(x, True))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [196]:
df_train.to_csv(
    '../data/obligations_fasttext_cls.txt',
    columns=['cleaned_sentence', 'label'],
    sep=' ',
    header=False,
    index=False,
    #escapechar='', quoting=csv.QUOTE_NONE
)

with open('../data/obligations_fasttext_cls.txt', 'r') as file:
    data = file.read()
    
data = re.sub("\"", '', data)

with open('../data/obligations_fasttext_cls.txt', 'w') as file:
    file.write(data)

In [200]:
model = fasttext.train_supervised('../data/obligations_fasttext_cls.txt')

In [198]:
preds, cofidence = model.predict(list(df_test.cleaned_sentence.values))
preds_bool = [re.sub('__label__', '', p[0]) == 'True' for p in preds]

In [113]:
# with preprocessing

print(classification_report(df_test.is_obligation.values, preds_bool))

              precision    recall  f1-score   support

       False       0.80      0.82      0.81      2113
        True       0.84      0.81      0.82      2371

    accuracy                           0.82      4484
   macro avg       0.82      0.82      0.82      4484
weighted avg       0.82      0.82      0.82      4484



In [120]:
# wo preprocessing

print(classification_report(df_test.is_obligation.values, preds_bool))

              precision    recall  f1-score   support

       False       0.80      0.81      0.81      2113
        True       0.83      0.82      0.83      2371

    accuracy                           0.82      4484
   macro avg       0.82      0.82      0.82      4484
weighted avg       0.82      0.82      0.82      4484



In [127]:
# with preprocessing, epochs 25

print(classification_report(df_test.is_obligation.values, preds_bool))

              precision    recall  f1-score   support

       False       0.79      0.80      0.80      2113
        True       0.82      0.81      0.82      2371

    accuracy                           0.81      4484
   macro avg       0.81      0.81      0.81      4484
weighted avg       0.81      0.81      0.81      4484



In [164]:
# with preprocessing, wordNgrams=2

print(classification_report(df_test.is_obligation.values, preds_bool))

              precision    recall  f1-score   support

       False       0.80      0.85      0.82      2113
        True       0.85      0.81      0.83      2371

    accuracy                           0.83      4484
   macro avg       0.83      0.83      0.83      4484
weighted avg       0.83      0.83      0.83      4484



In [201]:
# with preprocessing, stop words removed

print(classification_report(df_test.is_obligation.values, preds_bool))

              precision    recall  f1-score   support

       False       0.81      0.80      0.81      2113
        True       0.83      0.83      0.83      2371

    accuracy                           0.82      4484
   macro avg       0.82      0.82      0.82      4484
weighted avg       0.82      0.82      0.82      4484



In [199]:
# with preprocessing, stop words removed, ngram2

print(classification_report(df_test.is_obligation.values, preds_bool))

              precision    recall  f1-score   support

       False       0.81      0.80      0.81      2113
        True       0.83      0.83      0.83      2371

    accuracy                           0.82      4484
   macro avg       0.82      0.82      0.82      4484
weighted avg       0.82      0.82      0.82      4484

